In [2]:
# Liste der zu evaluierenden Modelle
mlist = [
"ProsusAI/finbert",
"cardiffnlp/twitter-roberta-base-sentiment-latest",
"nlptown/bert-base-multilingual-uncased-sentiment",
"siebert/sentiment-roberta-large-english",
]

# Imports
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import mlflow
import torch
import gc


# Data Import

#df = pd.read_csv('drive_download/classification_data_subset.csv')
df = pd.read_csv("drive_download/sentiment_data_financial.csv")
#df.kategorie_id = df.kategorie_id - 1
#df = df.rename(columns={"kategorie_id":"label", "inhalt":"text"})
#df = df.drop(columns=['kategorie_bezeichnung', 'titel', 'inhalt_kurz'])
df = df[df['text'].notna()]
traindf, testdf = train_test_split(df, test_size=0.2,stratify=df["label"])

train = Dataset.from_pandas(traindf)
test = Dataset.from_pandas(testdf)


dataset = DatasetDict()

dataset['train'] = train
dataset['test'] = test
device = "cuda" if torch.cuda.is_available() else "cpu"

model = ""
trainer =""

# Loop für Evaluierung der verschiedenen Modelle
for x in mlist:
  t = torch.cuda.get_device_properties(0).total_memory/1024
  r = torch.cuda.memory_reserved(0)/1024
  a = torch.cuda.memory_allocated(0)/1024
  f = r-a  # free inside reserved
  
  print(t,r,a,f)
  del model
  del trainer
  gc.collect()
  torch.cuda.empty_cache()
  torch.cuda.synchronize()
  print(t,r,a,f)

  mlflow.start_run()
  print(x)
  
  # Import Tokenizer
  from transformers import AutoTokenizer
  model_name = x

  tokenizer = AutoTokenizer.from_pretrained(model_name)
  

  def preprocess_function(examples):
      return tokenizer(examples["text"], truncation=True)

  tokenized_datasets = dataset.map(preprocess_function, batched=True)
  
  from transformers import DataCollatorWithPadding

  data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
  
  # Import Model
  from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

  model = AutoModelForSequenceClassification.from_pretrained(model_name, 
                                                            num_labels=3,
                                                            ignore_mismatched_sizes=True).to(device)
  
  # Metrik festsetzen
  import numpy as np
  from datasets import load_metric
  metric = load_metric("accuracy")

  def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
  
  num_epochs = 2

  args = TrainingArguments(
      output_dir ='./results',          
      num_train_epochs = num_epochs,
      per_device_train_batch_size = 16,   
      per_device_eval_batch_size = 16,   
      weight_decay = 0.01,               
      learning_rate = 2e-5,             
      metric_for_best_model = 'accuracy',    
      evaluation_strategy = "epoch",
      )


  trainer = Trainer(
      model,
      args,
      train_dataset=tokenized_datasets['train'],
      eval_dataset=tokenized_datasets['test'],
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics
  )  

  #Training + Tracking
  mlflow.autolog()
  trainer.train()
  mlflow.log_param("model",x)
  mlflow.end_run()

49853824.0 0.0 0.0 0.0
49853824.0 0.0 0.0 0.0
ProsusAI/finbert


2023-07-25 01:29:01.053182: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

loading configuration file config.json from cache at /home/rechcons/.cache/huggingface/hub/models--siebert--sentiment-roberta-large-english/snapshots/e78aa2b262b5f58e659fd765676b19c16a34f745/config.json
Model config RobertaConfig {
  "_name_or_path": "siebert/sentiment-roberta-large-english",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",


Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /home/rechcons/.cache/huggingface/hub/models--siebert--sentiment-roberta-large-english/snapshots/e78aa2b262b5f58e659fd765676b19c16a34f745/pytorch_model.bin
All model checkpoint weights were used when initializing RobertaForSequenceClassification.

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at siebert/sentiment-roberta-large-english and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([2, 1024]) in the checkpoint and torch.Size([3, 1024]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integration

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.414780,0.834711
2,No log,0.370013,0.866736


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 968
  Batch size = 64
/home/rechcons/PP3-NLP/.PPP-NLP-env/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: __index_level_0__, text. If __index_level_0__, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation ***